In [ ]:
import pandas as pd
import numpy as np
import math
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.datasets import make_regression
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_confusion_matrix



# We add all Plotly and Dash necessary librairies
import plotly.graph_objects as go
import dash
import dash_core_components as dcc
import dash_html_components as html
import dash_daq as daq
from dash.dependencies import Input, Output

In [ ]:
df = df= pd.read_csv("Data/predictive_maintainence.csv")
X= df.drop(columns=["process_id","failure"])
Y= df["failure"]


corr = X.corr()
mask = np.zeros_like(corr)
mask[np.triu_indices_from(mask)] = True
with sns.axes_style("white"):
    f, ax = plt.subplots(figsize=(100, 10))
    ax = sns.heatmap(corr, mask=mask,cmap='coolwarm', vmin=-1,vmax=1,annot=True, square=True)

cor_matrix = X.corr().abs()
upper_tri = cor_matrix.where(np.triu(np.ones(cor_matrix.shape),k=1).astype(np.bool))
to_drop = [column for column in upper_tri.columns if any(upper_tri[column] > 0.79)]
X = X.drop(columns=to_drop)
X_colm= X.columns
print(X_colm)


In [ ]:

df_feature_importances = pd.DataFrame(model.feature_importances_*100,columns=["Importance"],index=X_colm)
df_feature_importances = df_feature_importances.sort_values("Importance", ascending=False)


# We create a Features Importance Bar Chart
fig_features_importance = go.Figure()
fig_features_importance.add_trace(go.Bar(x=df_feature_importances.index,
                                         y=df_feature_importances["Importance"],
                                         marker_color='rgb(171, 226, 251)')
                                 )
fig_features_importance.update_layout(title_text='<b>Features Importance of the model<b>', title_x=0.5)
fig_features_importance.show()


In [171]:
# We record the name, min, mean and max of the three most important features
# We record the name, min, mean and max of the three most important features
dropdown_1_label = df_feature_importances.index[0]
dropdown_1_min = math.floor(df[dropdown_1_label].min())
dropdown_1_mean = round(df[dropdown_1_label].mean())
dropdown_1_max = round(df[dropdown_1_label].max())

dropdown_2_label = df_feature_importances.index[1]
dropdown_2_min = math.floor(df[dropdown_2_label].min())
dropdown_2_mean = round(df[dropdown_2_label].mean())
dropdown_2_max = round(df[dropdown_2_label].max())

dropdown_3_label = df_feature_importances.index[2]
dropdown_3_min = math.floor(df[dropdown_3_label].min())
dropdown_3_mean = round(df[dropdown_3_label].mean())
dropdown_3_max = round(df[dropdown_3_label].max())
dropdown_3_max

1921

In [179]:
app = dash.Dash()

# The page structure will be:
#    Features Importance Chart
#    <H4> Feature #1 name
#    Slider to update Feature #1 value
#    <H4> Feature #2 name
#    Slider to update Feature #2 value
#    <H4> Feature #3 name
#    Slider to update Feature #3 value
#    <H2> Updated Prediction
#    Callback fuction with Sliders values as inputs and Prediction as Output

# We apply basic HTML formatting to the layout
app.layout = html.Div(style={'textAlign': 'center', 'width': '800px', 'font-family': 'Verdana'},
                      
                    children=[

                        # Title display
                        html.H1(children="Simulation Tool"),
                        
                        # Dash Graph Component calls the fig_features_importance parameters
                        dcc.Graph(figure=fig_features_importance),
                        
                        # We display the most important feature's name
                        html.H4(children=dropdown_1_label),

                        # The Dash Slider is built according to Feature #1 ranges
                        dcc.Slider(
                            id='X1_slider',
                            min=dropdown_1_min,
                            max=dropdown_1_max,
                            step=0.5,
                            value=dropdown_1_mean,
                            marks={i: '{}'.format(i) for i in range(dropdown_1_min, dropdown_1_max+1,25)}
                            ),

                        html.H4(children=dropdown_2_label),

                        dcc.Slider(
                            id='X2_slider',
                            min=dropdown_2_min,
                            max=dropdown_2_max,
                            step=0.5,
                            value=dropdown_2_mean,
                            marks={i: '{}'.format(i) for i in range(dropdown_2_min, dropdown_2_max+1,10000)}
                        ),

                        html.H4(children=dropdown_3_label),

                        dcc.Slider(
                            id='X3_slider',
                            min=dropdown_3_min,
                            max=dropdown_3_max,
                            step=0.5,
                            value=dropdown_3_mean,
                            marks={i: '{}'.format(i) for i in range(dropdown_3_min, dropdown_3_max+1,300)}
                        ),

                        html.H2(id="prediction_result")

                    ])


In [181]:
# The callback function will provide one "Ouput" in the form of a string (=children)
@app.callback(Output(component_id="prediction_result",component_property="children"),
# The values correspnding to the three sliders are obtained by calling their id and value property
              [Input("X1_slider","value"), Input("X2_slider","value"), Input("X3_slider","value")])
      
def update_prediction(X1, X2, X3):
    # We create a NumPy array in the form of the original features
    # ["Pressure","Viscosity","Particles_size", "Temperature","Inlet_flow", "Rotating_Speed","pH","Color_density"]
    # Except for the X1, X2 and X3, all other non-influencing parameters are set to their mean
    input_X= np.array([df['min_x_position'].mean(),df['avg_x_position'].mean(), df['avg_y_position'].mean(),df['avg_main_current'].mean(), df['max_noise'].mean(), df['avg_vibrations'].mean(), df['max_vibrations'].mean(),df['avg_z_position'].mean(), df['min_z_position'].mean(), df['penetration'].mean(), df['process_time'].mean(),X1, X2, df['luminosity'].mean(), X3,df['avg_main_force'].mean(),df['x_distance'].mean(), df['y_distance'].mean(), df['z_distance'].mean(), df['time_reference_diff'].mean(), df['x_pos_reference_diff'].mean(), df['z_pos_reference_diff'].mean(),df['off_limits_average'].mean(), df['position_range'].mean(), df['current_range'].mean(),df['current_signal_noise_ratio'].mean(), df['preparation_time'].mean]).reshape(1,-1)

    # Prediction is calculated based on the input_X array
    prediction = model.predict(input_X)[0]
    
    # And retuned to the Output of the callback function
    return "Prediction: {}".format(round(prediction,1))


In [ ]:
if __name__ == "__main__":
    app.run_server()

In [196]:
input_X= pd.DataFrame([df['min_x_position'].mean(),df['avg_x_position'].mean(), df['avg_y_position'].mean(),df['avg_main_current'].mean(), df['max_noise'].mean(), df['avg_vibrations'].mean(), df['max_vibrations'].mean(),df['avg_z_position'].mean(), df['min_z_position'].mean(), df['penetration'].mean(), df['process_time'].mean(),5, 6, df['luminosity'].mean(), 7,df['avg_main_force'].mean(),df['x_distance'].mean(), df['y_distance'].mean(), df['z_distance'].mean(), df['time_reference_diff'].mean(), df['x_pos_reference_diff'].mean(), df['z_pos_reference_diff'].mean(),df['off_limits_average'].mean(), df['position_range'].mean(), df['current_range'].mean(),df['current_signal_noise_ratio'].mean(), df['preparation_time'].mean()])
input_X
prediction = model.predict(input_X)
#prediction

ValueError: Feature shape mismatch, expected: 27, got 1